In [1]:
## Import all the packages and library
import pandas as pd
import warnings
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Ignore all warnings
warnings.filterwarnings('ignore')

In [2]:
## Load both the data sources
users_data = pd.read_csv('data/users_data.csv')
transaction_data = pd.read_csv('data/transactions_data.csv')

In [3]:
## Add new columns and transform transaction_data

# Convert 'date' column to datetime format for easier date manipulation
transaction_data['date'] = pd.to_datetime(transaction_data['date'])

# Remove '$' symbols and commas from the 'amount' column and convert it to float
transaction_data['amount'] = transaction_data['amount'].replace({'\$': '', ',': ''}, regex=True).astype(float)

# Extract the month and year from the 'date' column and create new columns
transaction_data['month'] = transaction_data['date'].dt.month  # Create 'month' column
transaction_data['year'] = transaction_data['date'].dt.year    # Create 'year' column

# Create a new column 'transaction_type' based on the 'use_chip' column
# Assign 'Chip' for 'Chip Transaction' and 'Non-Chip' for other values
transaction_data['transaction_type'] = transaction_data['use_chip'].apply(
    lambda x: 'Chip' if x == 'Chip Transaction' else 'Non-Chip'
)

In [4]:
## Add new columns and transform users_data
# Remove the '$' from 'yearly_income' and convert the column to a numeric float type
users_data['yearly_income'] = users_data['yearly_income'].replace({'\$': '', ',': ''}, regex=True).astype(float)

# Remove the '$' from 'total_debt' and convert the column to a numeric float type
users_data['total_debt'] = users_data['total_debt'].replace({'\$': '', ',': ''}, regex=True).astype(float)

# Remove the '$' from 'per_capita_income' and convert the column to a numeric float type
users_data['per_capita_income'] = users_data['per_capita_income'].replace({'\$': '', ',': ''}, regex=True).astype(float) 

In [5]:
## check unique number if users
print (transaction_data['client_id'].nunique())

# null values across columns
null_columns = transaction_data.isnull().sum()

# Print results
print("Columns with null values:")
print(null_columns[null_columns > 0])

1206
Columns with null values:
Series([], dtype: int64)


In [6]:
## check unique number if users
print (users_data['id'].nunique())

# null values across columns
null_columns = users_data.isnull().sum()

# Print results
print("Columns with null values:")
print(null_columns[null_columns > 0])

2000
Columns with null values:
Series([], dtype: int64)


## Feature Engineering

We will utilize transaction data to derive multiple features that capture individual user behavior based on their transaction history. Each feature type is designed to provide specific insights, aiding in the clustering of different user segments effectively


## Static Features

Calculates key transaction features for each client. It computes total spending (`total_spending`), average transaction value (`avg_transaction`), and transaction frequency (`transaction_count`). Additionally, it analyzes transaction types (Chip vs. Non-Chip) by grouping and unstacking their frequencies. All features are consolidated into a single DataFrame for client-level insights

In [7]:
# Calculate total spending per client_id
total_spending = transaction_data.groupby('client_id')['amount'].sum().reset_index()
total_spending.rename(columns={'amount': 'total_spending'}, inplace=True)  # Rename the column to 'total_spending'

# Calculate average transaction value per client_id
avg_transaction = transaction_data.groupby('client_id')['amount'].mean().reset_index()
avg_transaction.rename(columns={'amount': 'avg_transaction'}, inplace=True)  # Rename the column to 'avg_transaction'

# Calculate transaction frequency (number of transactions) per client_id
transaction_freq = transaction_data.groupby('client_id')['id'].count().reset_index()
transaction_freq.rename(columns={'id': 'transaction_count'}, inplace=True)  # Rename the column to 'transaction_count'

# Calculate transaction frequency across transaction types (chip and non-chip)
chip_transaction = transaction_data.groupby(['client_id', 'use_chip'])['id'].count().unstack(fill_value=0).reset_index()

# Merge all calculated features into a single DataFrame for client_id level analysis
customer_features = total_spending.merge(avg_transaction, on='client_id')  # Merge total spending and average transaction value
customer_features = customer_features.merge(transaction_freq, on='client_id')  # Merge transaction frequency
customer_features = customer_features.merge(chip_transaction, on='client_id')  # Merge chip-based transaction frequency

In [8]:
## Display top 2 rows to verify the output
customer_features.head(2)

,client_id,total_spending,avg_transaction,transaction_count,Chip Transaction,Online Transaction,Swipe Transaction
0,0,50852.09,47.703649,1066,958,89,19
1,1,29110.22,35.894229,811,0,327,484


## Rolling Features for Transaction Behavior Analysis

This code computes rolling features to analyze user's past transaction behaviors over 3, 6, and 9 months. It calculates the sum, average, maximum, and minimum spending for these periods, as well as the rate of change in spending compared to the latest month. Also measures spending volatility (standard deviation) to capture fluctuations

In [9]:
# Aggregate spending per month for each client_id
monthly_spending = transaction_data.groupby(['client_id', 'month'])['amount'].sum().reset_index()

# Pivot to create a matrix of monthly spending
spending_matrix = monthly_spending.pivot(index='client_id', columns='month', values='amount').fillna(0)

In [10]:
# Create a new DataFrame for rolling features to capture users' past transaction behaviors
rolling_features = pd.DataFrame(index=spending_matrix.index)

# Calculate past 3, 6, and 9 months spending (sum)
rolling_features['past_3_months_sum'] = spending_matrix.rolling(window=3, axis=1).sum().iloc[:, -1]
rolling_features['past_6_months_sum'] = spending_matrix.rolling(window=6, axis=1).sum().iloc[:, -1]
rolling_features['past_9_months_sum'] = spending_matrix.rolling(window=9, axis=1).sum().iloc[:, -1]

# Calculate past 3, 6, and 9 months spending (average)
rolling_features['past_3_months_avg'] = spending_matrix.rolling(window=3, axis=1).mean().iloc[:, -1]
rolling_features['past_6_months_avg'] = spending_matrix.rolling(window=6, axis=1).mean().iloc[:, -1]
rolling_features['past_9_months_avg'] = spending_matrix.rolling(window=9, axis=1).mean().iloc[:, -1]

# Calculate maximum and minimum spending in the past 3, 6, and 9 months
rolling_features['past_3_months_max'] = spending_matrix.rolling(window=3, axis=1).max().iloc[:, -1]
rolling_features['past_3_months_min'] = spending_matrix.rolling(window=3, axis=1).min().iloc[:, -1]
rolling_features['past_6_months_max'] = spending_matrix.rolling(window=6, axis=1).max().iloc[:, -1]
rolling_features['past_6_months_min'] = spending_matrix.rolling(window=6, axis=1).min().iloc[:, -1]
rolling_features['past_9_months_max'] = spending_matrix.rolling(window=9, axis=1).max().iloc[:, -1]
rolling_features['past_9_months_min'] = spending_matrix.rolling(window=9, axis=1).min().iloc[:, -1]

# Calculate the rate of change in spending compared to the lastest month
rolling_features['rate_of_change_3m'] = (rolling_features['past_3_months_sum'] - spending_matrix.iloc[:, -1]) / spending_matrix.iloc[:, -1]
rolling_features['rate_of_change_6m'] = (rolling_features['past_6_months_sum'] - spending_matrix.iloc[:, -1]) / spending_matrix.iloc[:, -1]
rolling_features['rate_of_change_9m'] = (rolling_features['past_9_months_sum'] - spending_matrix.iloc[:, -1]) / spending_matrix.iloc[:, -1]

# Calculate the standard deviation (volatility) of spending over the last 3, 6, and 9 months
rolling_features['spending_volatility_3m'] = spending_matrix.rolling(window=3, axis=1).std().iloc[:, -1]
rolling_features['spending_volatility_6m'] = spending_matrix.rolling(window=6, axis=1).std().iloc[:, -1]
rolling_features['spending_volatility_9m'] = spending_matrix.rolling(window=9, axis=1).std().iloc[:, -1]

# Fill missing values with 0 for cases where there isn't enough data for rolling calculations
rolling_features.fillna(0, inplace=True)

In [11]:
# Replace inf or -inf values with NaN or another default value (e.g., 0)
rolling_features['rate_of_change_3m'] = rolling_features['rate_of_change_3m'].replace([np.inf, -np.inf], np.nan)
rolling_features['rate_of_change_6m'] = rolling_features['rate_of_change_6m'].replace([np.inf, -np.inf], np.nan)
rolling_features['rate_of_change_9m'] = rolling_features['rate_of_change_9m'].replace([np.inf, -np.inf], np.nan)


# Fill missing values with 0 for cases where there isn't enough data for rolling calculations
rolling_features.fillna(0, inplace=True)

In [12]:
## Display top 2 rows
rolling_features.head(2)

,past_3_months_sum,past_6_months_sum,past_9_months_sum,past_3_months_avg,past_6_months_avg,past_9_months_avg,past_3_months_max,past_3_months_min,past_6_months_max,past_6_months_min,past_9_months_max,past_9_months_min,rate_of_change_3m,rate_of_change_6m,rate_of_change_9m,spending_volatility_3m,spending_volatility_6m,spending_volatility_9m
client_id,,,,,,,,,,,,,,,,,,
0,15724.66,28900.12,43981.00,5241.553333,4816.686667,4886.777778,5952.93,3985.09,5952.93,3985.09,5952.93,3985.09,2.945873,6.252062,10.036388,1091.301137,852.178344,699.248083
1,9740.26,17605.51,25469.33,3246.753333,2934.251667,2829.925556,4931.47,2332.39,4931.47,2332.39,4931.47,2171.76,3.176085,6.548270,9.919842,1460.783148,991.133033,826.691997


In [13]:
## Merge 2 dataframes; combine all the features to customer_features
customer_features = customer_features.merge(rolling_features, on='client_id')

## Merchant Spending Patterns

Analyzes user spending across merchants, focusing on:

1. **Top Categories**: Identifies the top 5 merchant categories for each client based on spending
2. **Category Diversity**: Counts unique merchant categories to measure spending variety
3. **Unique Merchants**: Tracks the number of distinct merchants per client

In [14]:
## Determine the top 5 categories for each client_id based on their spending
# Group by client_id and merchant_category, summing the amounts
top_categories = transaction_data.groupby(['client_id', 'merchant_category'])['amount'].sum().reset_index()

# Sort values by client_id and amount
top_categories_sorted = top_categories.sort_values(by=['client_id', 'amount'], ascending=[True, False])

# For each client_id, get the top 5 merchant categories
top_5_categories = top_categories_sorted.groupby('client_id').head(5)

# Create a list of top categories for each client_id
top_5_categories_list = top_5_categories.groupby('client_id')['merchant_category'].apply(list).reset_index()

# Final df
top_5_categories_list.rename(columns={'merchant_category': 'top_5_merchant_categories'}, inplace=True)

In [15]:
## Count the number of distinct merchant categories for each client, which indicates how diverse their spending is across different categories
category_diversity = transaction_data.groupby('client_id')['merchant_category'].nunique().reset_index()
category_diversity.rename(columns={'merchant_category': 'category_diversity'}, inplace=True)

In [16]:
## Count the number of unique merchants for each client to see how many different places they shop or spend money
unique_merchants = transaction_data.groupby('client_id')['merchant_id'].nunique().reset_index()
unique_merchants.rename(columns={'merchant_id': 'num_unique_merchants'}, inplace=True)

In [17]:
## Merge all the dataframes for merchant details
unique_merchants = unique_merchants.merge(category_diversity, on='client_id')
unique_merchants = unique_merchants.merge(top_5_categories_list, on='client_id')

In [18]:
## Merge 2 dataframes
customer_features = customer_features.merge(unique_merchants, on='client_id')

In [19]:
## Display top 2 rows
customer_features.head(2)

,client_id,total_spending,avg_transaction,transaction_count,Chip Transaction,Online Transaction,Swipe Transaction,past_3_months_sum,past_6_months_sum,past_9_months_sum,...,past_9_months_min,rate_of_change_3m,rate_of_change_6m,rate_of_change_9m,spending_volatility_3m,spending_volatility_6m,spending_volatility_9m,num_unique_merchants,category_diversity,top_5_merchant_categories
0,0,50852.09,47.703649,1066,958,89,19,15724.66,28900.12,43981.00,...,3985.09,2.945873,6.252062,10.036388,1091.301137,852.178344,699.248083,134,51,"[Drinking Places (Alcoholic Beverages), Utilit..."
1,1,29110.22,35.894229,811,0,327,484,9740.26,17605.51,25469.33,...,2171.76,3.176085,6.548270,9.919842,1460.783148,991.133033,826.691997,77,43,"[Taxicabs and Limousines, Telecommunication Se..."


## Transaction Timing Analysis

1. **Data Preparation**: Transactions are sorted by `client_id` and `date`
2. **Time Difference**: Calculates the time difference between consecutive transactions for each client
3. **Average Time**: Computes the average time between transactions (in seconds) for each client

In [20]:
# Sort the data by client_id and date
transaction_data = transaction_data.sort_values(by=['client_id', 'date'])

# Calculate the time difference between consecutive transactions (in seconds, for example)
transaction_data['time_diff'] = transaction_data.groupby('client_id')['date'].diff().dt.total_seconds()

# Calculate the average time difference (in seconds) for each client_id
average_time_between_transactions = transaction_data.groupby('client_id')['time_diff'].mean().reset_index()

# Rename the column for clarity
average_time_between_transactions.rename(columns={'time_diff': 'avg_time_between_transactions_seconds'}, inplace=True)

In [21]:
## Merge 2 dataframes
customer_features = customer_features.merge(average_time_between_transactions, on='client_id')

In [22]:
## Display top 2 rows
customer_features.head(2)

,client_id,total_spending,avg_transaction,transaction_count,Chip Transaction,Online Transaction,Swipe Transaction,past_3_months_sum,past_6_months_sum,past_9_months_sum,...,rate_of_change_3m,rate_of_change_6m,rate_of_change_9m,spending_volatility_3m,spending_volatility_6m,spending_volatility_9m,num_unique_merchants,category_diversity,top_5_merchant_categories,avg_time_between_transactions_seconds
0,0,50852.09,47.703649,1066,958,89,19,15724.66,28900.12,43981.00,...,2.945873,6.252062,10.036388,1091.301137,852.178344,699.248083,134,51,"[Drinking Places (Alcoholic Beverages), Utilit...",24624.394366
1,1,29110.22,35.894229,811,0,327,484,9740.26,17605.51,25469.33,...,3.176085,6.548270,9.919842,1460.783148,991.133033,826.691997,77,43,"[Taxicabs and Limousines, Telecommunication Se...",32340.592593


## Spending Patterns indicators and Behaviors

1. **Spending Consistency**: Proportion of spending in the last 3, 6, and 9 months relative to total spending (`spending_consistency_3m`, `spending_consistency_6m`, `spending_consistency_9m`).

2. **Payment Method Preference**: Ratio of transactions made via Chip, Online, and Swipe methods to the total transactions (`chip_transaction_ratio`, `online_transaction_ratio`, `swipe_transaction_ratio`).

3. **Absolute Changes**: Difference in spending between recent and earlier periods:
   - 3 months vs. 6 months (`absolute_change_3m_vs_6m`)
   - 6 months vs. 9 months (`absolute_change_6m_vs_9m`).

4. **Growth Rates**: Percentage growth in spending:
   - 3 months vs. earlier 3 months (`growth_rate_3m_vs_6m`).
   - 6 months vs. earlier 6 months (`growth_rate_6m_vs_9m`).

5. **Spending Proportions**: Fraction of spending in recent periods compared to earlier periods:
   - Last 3 months vs. 6 months (`proportion_recent_3m_vs_6m`).
   - Last 6 months vs. 9 months (`proportion_recent_6m_vs_9m`).


In [23]:
## Create empty dataframe
spending_pattern = pd.DataFrame()
spending_pattern['client_id'] = customer_features['client_id']

## Spending in the last 3, 6, 9 months to the total spending. This indicates how much of a client’s total 
## spending is recent
spending_pattern['spending_consistency_3m'] = customer_features['past_3_months_sum'] / customer_features['total_spending']
spending_pattern['spending_consistency_6m'] = customer_features['past_6_months_sum'] / customer_features['total_spending']
spending_pattern['spending_consistency_9m'] = customer_features['past_9_months_sum'] / customer_features['total_spending']

# Payment Method Preference
spending_pattern['chip_transaction_ratio'] = customer_features['Chip Transaction'] / customer_features['transaction_count']
spending_pattern['online_transaction_ratio'] = customer_features['Online Transaction'] / customer_features['transaction_count']
spending_pattern['swipe_transaction_ratio'] = customer_features['Swipe Transaction'] /customer_features['transaction_count']

## Absolute change in difference between the two periods
spending_pattern['absolute_change_3m_vs_6m'] = customer_features['past_3_months_sum'] - (customer_features['past_6_months_sum'] - customer_features['past_3_months_sum'])
spending_pattern['absolute_change_6m_vs_9m'] = customer_features['past_6_months_sum'] - (customer_features['past_9_months_sum'] - customer_features['past_6_months_sum'])

## Growth rate compares the magnitude of the recent period to the earlier period in %
earlier_3_months = customer_features['past_6_months_sum'] - customer_features['past_3_months_sum']
spending_pattern['growth_rate_3m_vs_6m'] = (customer_features['past_3_months_sum'] / earlier_3_months) - 1

earlier_6_months = customer_features['past_9_months_sum'] - customer_features['past_6_months_sum']
spending_pattern['growth_rate_6m_vs_9m'] = (customer_features['past_6_months_sum'] / earlier_6_months) - 1

## Fraction of Spend over months intervals
spending_pattern['proportion_recent_3m_vs_6m'] = customer_features['past_3_months_sum'] / customer_features['past_6_months_sum']
spending_pattern['proportion_recent_6m_vs_9m'] = customer_features['past_6_months_sum'] / customer_features['past_9_months_sum']


In [24]:
## Merge 2 dataframes
customer_features = customer_features.merge(spending_pattern, on='client_id')

In [25]:
## Display top 2 rows
customer_features.tail(2)

,client_id,total_spending,avg_transaction,transaction_count,Chip Transaction,Online Transaction,Swipe Transaction,past_3_months_sum,past_6_months_sum,past_9_months_sum,...,spending_consistency_9m,chip_transaction_ratio,online_transaction_ratio,swipe_transaction_ratio,absolute_change_3m_vs_6m,absolute_change_6m_vs_9m,growth_rate_3m_vs_6m,growth_rate_6m_vs_9m,proportion_recent_3m_vs_6m,proportion_recent_6m_vs_9m
1204,1997,46296.87,54.724433,846,692,40,114,14196.32,33135.04,43441.84,...,0.938332,0.817967,0.047281,0.134752,-4742.40,22828.24,-0.250408,2.214872,0.428438,0.762745
1205,1998,7047.76,18.794027,375,328,19,28,2289.70,4644.22,5918.70,...,0.839799,0.874667,0.050667,0.074667,-64.82,3369.74,-0.027530,2.644012,0.493021,0.784669


## Merge with users_data and add some more derived features to the final dataframe

In [26]:
## Rename id column in users_data to client_id
users_data.rename(columns={'id': 'client_id'}, inplace=True)
users_data.head(2)

,client_id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278.0,59696.0,127613.0,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,37891.0,77254.0,191349.0,701,5


In [27]:
## Create Debt-to-Income (DTI) ratio column which could be helpful in defining the user financial habits
users_data['debt_to_income_ratio'] = users_data['total_debt'] / users_data['yearly_income']

## Merge transaction data features with the users_data 

In [28]:
## Merge 2 dataframe to get the final df which be used for customer segmentation 
customer_features = customer_features.merge(users_data, on='client_id',how='left')

In [32]:
import os

# Define the output folder path
output_folder = "output"

# Check if the folder exists; if not, create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

print(f"Output folder '{output_folder}' is ready.")

Output folder 'output' is ready.


In [33]:
## Save df as csv for clustering 
customer_features.to_csv('output/Final_transaction_derived_features_across_users.csv',index=False)